# Maryland Dam Design Storm Data

Version 2.0
March 18, 2024
Async enabled

1. Export the Dams Master Table as `.xlsx`, then upload it here.
2. Specify a design storm for each dam as desired. If not, it will default to the 6-hour, 100-year storm.
3. This program will pull Atlas-14 rainfall in inches for each dam, and save it to a `.csv` file.

Notes:

1. Confirm all dams in the Master Table have a `'Latitude'` and `'Longitude'` value
2. Once you receive the outputted `.csv`, rename and/or download it to prevent its getting overwritten on the next run.

In [1]:
import datetime as dt
import pytz
import requests
import matplotlib.pyplot as plt
import pandas as pd
import json
import requests
import re
import requests
from requests.packages import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Here is where you put the filepath of the Dams Master Table!
uploaded = "C:/Users/aben-sorek/Desktop/weather code/Dams Master Table.xlsx"
damsMasterTable = pd.read_excel(uploaded)

In [3]:
# Create a dataframe with the Excel data
dams = pd.read_excel(uploaded)

# Any dam without a Design Storm Duration or Interval will be assigned the defaults here
dams['Duration'] = '6-hr'
dams['Interval'] = '100'

In [4]:
# This test code was used during development.
# If you want to test the code without using an .xlsx, use this cell rather than
# the preceding cell.

#dnames = ['Blue','Yellow','Red']
#dDesignStorms = [100,100,1000]
#dCoordinates = [[39,-76],[38,-77],[37,-78]]
#dStormDuration = ['6-hr','6-hr','6-hr']
#dStormInterval = ['100','100','100']

#data = {'Name':dnames,'Coordinates':dCoordinates, 'Duration':dStormDuration, 'Interval':dStormInterval}

In [5]:
def create_dataframe_from_NOAAstring(data_string):
    pd.options.mode.chained_assignment = None
    lines = data_string.strip().split('\n')
    data = [line.split(',') for line in lines]

    # As of August 2023, cleaning NOAA data means: removing extraneous
    # rows from the top and bottom; removing some characters from
    # the start and end of the first and last cells, respectively; removing any
    # remaining square brackets; and renaming the row and column headers.
    df = pd.DataFrame(data)
    df2 = df.head(-17)
    df3 = df2.tail(-1)
    df3[df3.columns[-1]] = (df3[df3.columns[-1]].str[:-2])
    df3[df3.columns[0]] = (df3[df3.columns[0]].str[13:])

    # The data is now all in one row. Rework it into rows and columns
    data_string = df3.to_string(header=False, index=False, index_names=False)

    # Extract individual arrays using regex
    arrays = re.findall(r'\[.*?\]', data_string)

    # Remove whitespace and quotes, and split each array into a list
    data_list = [array.replace("'", "").split() for array in arrays]

    # Create a Pandas DataFrame from the list of lists
    df4 = pd.DataFrame(data_list)

    # Remove the '[' and ']' characters from the dataframe
    df5 = df4.applymap(lambda x: x.replace("[", "").replace("]", ""))
    df5.columns=['1','2','5','10','25','50','100','200','500','1000']
    df5.index=['5-min','10-min','15-min','30-min','60-min','2-hr','3-hr','6-hr','12-hr','24-hr','2-day','3-day','4-day','7-day','10-day','20-day','30-day','45-day','60-day']

    return df5

if __name__ == "__main__":
    input_data = """
    Value1,Value2,Value3
    1,2,3
    4,5,6
    7,8,9
    """

In [6]:
import aiohttp
import asyncio
from pandas import DataFrame

async def fetch_data(session, atlasURL, dams, dam):
    url = f"{atlasURL}?lat={dams['Latitude'][dam]}&lon={dams['Longitude'][dam]}&type=pf&data=depth&units=english&series=pds"
    async with session.get(url, ssl=False) as response:
        return await response.text()

async def fetch_all_data(atlasURL, dams):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for dam in range(len(dams)):
            task = asyncio.create_task(fetch_data(session, atlasURL, dams, dam))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        return responses

async def main():
    atlasURL = 'https://hdsc.nws.noaa.gov/cgi-bin/hdsc/new/cgi_readH5.py'
    responses = await fetch_all_data(atlasURL, dams)

    ser = []
    for dam, response in enumerate(responses):
        ser.append(create_dataframe_from_NOAAstring(response).loc[dams.loc[dam, 'Duration'], dams.loc[dam, 'Interval']])
        if (dam + 1) % 5 == 0 or dam == 0 or dam == len(dams):
            print(f'Dam {dam + 1} of {len(dams) + 1}. {round((dam + 1) / (len(dams) + 1) * 100, 2)}% completed.')

    dams['designPrecip'] = ser
    return dams  # Return the modified DataFrame

# Run the event loop in the Jupyter environment
dams_with_design_precip = await main()
print(dams_with_design_precip)

Dam 1 of 638. 0.16% completed.
Dam 5 of 638. 0.78% completed.


Dam 10 of 638. 1.57% completed.
Dam 15 of 638. 2.35% completed.
Dam 20 of 638. 3.13% completed.


Dam 25 of 638. 3.92% completed.
Dam 30 of 638. 4.7% completed.


Dam 35 of 638. 5.49% completed.
Dam 40 of 638. 6.27% completed.
Dam 45 of 638. 7.05% completed.
Dam 50 of 638. 7.84% completed.


Dam 55 of 638. 8.62% completed.
Dam 60 of 638. 9.4% completed.
Dam 65 of 638. 10.19% completed.


...

Dam 580 of 638. 90.91% completed.
Dam 585 of 638. 91.69% completed.
Dam 590 of 638. 92.48% completed.


Dam 595 of 638. 93.26% completed.
Dam 600 of 638. 94.04% completed.


Dam 605 of 638. 94.83% completed.
Dam 610 of 638. 95.61% completed.
Dam 615 of 638. 96.39% completed.


Dam 620 of 638. 97.18% completed.
Dam 625 of 638. 97.96% completed.


Dam 630 of 638. 98.75% completed.
Dam 635 of 638. 99.53% completed.
    Inspector Staff_2021 Staff_2019 Staff_2017  Entered ETS FY24  \
0         ASR        ASR        ASR        ASR             False   
1         ASR        ASR        ASR        ASR             False   
2         ASR        ASR        ASR        ASR             False   
3         WSA        SEB        SEB        BWH             False   
4         ASR        ASR        SEB        HVA             False   
..        ...        ...        ...        ...               ...   
632       VPD        KRF        NaN        NaN             False   
633       ASR        NaN        NaN        NaN             False   
634       NaN        NaN        NaN        NaN             False   
635       SEB        NaN        NaN        NaN             False   
636       SEB        NaN        NaN        NaN             False   

     Entered ETS FY23  TEMPO FY22  Entered TEMPO FY21  Entered TEMPO FY20  \
0               False       False     

In [7]:
print(type(dams['designPrecip']))
print((dams['designPrecip']))

<class 'pandas.core.series.Series'>
0      5.46
1      5.64
2      5.39
3      3.83
4      5.41
       ... 
632    5.56
633    5.34
634    5.42
635    5.39
636    5.41
Name: designPrecip, Length: 637, dtype: object


In [8]:
dams.to_csv('damsNOAA.csv', index=False)
